In [1]:
import json
import numpy as np
from tqdm import tqdm
from collections import Counter

In [2]:
from gensim.models import word2vec
from gensim import models
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
from lexrank import LexRank

In [3]:
pre_data_PATH = './data/'

In [4]:
with open(pre_data_PATH+'com_product_intro_word_articles_test.json') as jsonfile:
    com_product_intro_word = json.load(jsonfile)
com_product_intro_word

[['椰林 遍佈 島 美托三角洲 離 胡志明市 段 距離 但 許多 遊客 深受 那裡 水鄉澤國 傳統 集市 的 吸引 願意 驅車 前往 ',
  '胡志明市 出發 便利 接送 服務 你 少 交通 的 煩惱 ',
  '我們 搭上 小 船 入坐 馬車 到 濃郁 風情 湄公河 市鎮 一探 吧 ',
  '來到 越南 胡志明市 預訂 印度 支那號 遊船 晚宴 共有 種 套餐 供 選擇 讓 在 欣賞 西貢河 沿岸 夜景 之餘 放鬆 享用 精緻 晚宴 美食 遊船 更 精彩 舞蹈 民謠 音樂 傳統 表演 帶給 胡志明 浪漫 夜晚 特別 體驗 ',
  '胡志明 古芝 地道 私人 小團 日 遊 搭乘 舒適 空調 專車 前往 越南 戰爭 時期 重要 遺跡 全程 專業 中文 英語 導遊 隨行 導覽 帶 仔細 認識 古芝 地道 歷史 背景 適合 人 一同 搭 車 出遊 親身 感受 越戰 時期 緊張 氣氛 ',
  '越南  Viettel  上網 電話卡 前  8 GB  高速 上網 流量 超過 後 降速  64 kbps 效期  15 天 再 加贈 免費 接聽 來電 適用 越南 全 境 河內 胡志明市 重點 觀光區 皆 使用 ， 方便 旅途 中 隨時 卡 上網 ',
  '想 越南 走跳 能 少 最 便宜 最 方便 背包客 必備 交通 方式  Open Bus ',
  '事先 預定 巴士 的 座位 不用 害怕 熱門 時段 不了 車 '],
 ['個 小時 穿越 泰國 古今 了 解 泰國 豐厚 迷人 歷史 文化 ',
  '專業 演員 配合 絢爛 舞台 效果 曼谷暹羅 天使 劇場 你 重新 認識 泰國 ',
  '還可以 開場 前 欣賞 戶外 表演 參觀 民俗 展區 享用 自助 晚餐 品嘗 道地 泰國 美食 ',
  '專業 貼心 中文 導遊 陪伴 下 體驗 傳統 市集 鐵道 共存 奇景 美功 鐵道 市集 暢遊 百 年 歷史 丹嫩莎朵 水 市場 看看 式 特色 小吃 個性 商店 安帕瓦 水上 市場 參觀 獨具 特色 百 年 基督教堂 參訪 泰國 十 靈廟 一 「 樹中廟 」 並 唯美 夜色 中 迎 微風 ， 乘船 觀賞 閃爍 飛舞 螢火蟲 ',
  'Tripadvisor 顆 星 滿分 評價 曼谷泰式 烹飪 教室 乘船 觀賞 昭披 耶河 風光 後 抵達 綠意盎然 私人 香草 花園 

## LexRank

### Build Graph

In [7]:
similarity_matrix_list = []
for i in tqdm(range(len(com_product_intro_word))):
    lxr = LexRank(com_product_intro_word)
    tf_scores = [Counter(sent.split()) for sent in com_product_intro_word[i]]
    similarity_matrix = lxr._calculate_similarity_matrix(tf_scores)
    for j in range(len(similarity_matrix)):
        similarity_matrix[j][j] = 0
    similarity_matrix_list.append(similarity_matrix)

100%|██████████| 174/174 [05:54<00:00,  2.04s/it]


[array([[0.        , 0.15122737, 0.        , 0.05359518, 0.00109387,
         0.04450084, 0.        , 0.00222882],
        [0.15122737, 0.        , 0.        , 0.14261965, 0.        ,
         0.13448279, 0.15818187, 0.00673554],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        ],
        [0.05359518, 0.14261965, 0.        , 0.        , 0.09231041,
         0.10214837, 0.05091473, 0.        ],
        [0.00109387, 0.        , 0.        , 0.09231041, 0.        ,
         0.05200761, 0.04476923, 0.01387705],
        [0.04450084, 0.13448279, 0.        , 0.10214837, 0.05200761,
         0.        , 0.1014619 , 0.        ],
        [0.        , 0.15818187, 0.        , 0.05091473, 0.04476923,
         0.1014619 , 0.        , 0.        ],
        [0.00222882, 0.00673554, 0.        , 0.        , 0.01387705,
         0.        , 0.        , 0.        ]]),
 array([[0.        , 0.1405005 , 0.1625084 , 0.07887851, 0.09078633,
         

In [11]:
ranked_sentence_list = []
for i in tqdm(range(len(com_product_intro_word))):
    lex_scores = lxr.rank_sentences(com_product_intro_word[i], threshold=None)
    zipped = zip(lex_scores, com_product_intro_word[i])
    ranked_sentence = sorted(zipped, key = lambda x: x[0], reverse = True)
    ranked_sentence_list.append(ranked_sentence)
ranked_sentence_list

100%|██████████| 174/174 [00:02<00:00, 73.87it/s]


[[(1.1970113124339734, '胡志明市 出發 便利 接送 服務 你 少 交通 的 煩惱 '),
  (1.1313532842807295,
   '越南  Viettel  上網 電話卡 前  8 GB  高速 上網 流量 超過 後 降速  64 kbps 效期  15 天 再 加贈 免費 接聽 來電 適用 越南 全 境 河內 胡志明市 重點 觀光區 皆 使用 ， 方便 旅途 中 隨時 卡 上網 '),
  (1.0770247280516076,
   '來到 越南 胡志明市 預訂 印度 支那號 遊船 晚宴 共有 種 套餐 供 選擇 讓 在 欣賞 西貢河 沿岸 夜景 之餘 放鬆 享用 精緻 晚宴 美食 遊船 更 精彩 舞蹈 民謠 音樂 傳統 表演 帶給 胡志明 浪漫 夜晚 特別 體驗 '),
  (1.0231365901672222, '想 越南 走跳 能 少 最 便宜 最 方便 背包客 必備 交通 方式  Open Bus '),
  (1.0, '我們 搭上 小 船 入坐 馬車 到 濃郁 風情 湄公河 市鎮 一探 吧 '),
  (0.9269512700562872,
   '椰林 遍佈 島 美托三角洲 離 胡志明市 段 距離 但 許多 遊客 深受 那裡 水鄉澤國 傳統 集市 的 吸引 願意 驅車 前往 '),
  (0.8938881162505237,
   '胡志明 古芝 地道 私人 小團 日 遊 搭乘 舒適 空調 專車 前往 越南 戰爭 時期 重要 遺跡 全程 專業 中文 英語 導遊 隨行 導覽 帶 仔細 認識 古芝 地道 歷史 背景 適合 人 一同 搭 車 出遊 親身 感受 越戰 時期 緊張 氣氛 '),
  (0.7506346987595897, '事先 預定 巴士 的 座位 不用 害怕 熱門 時段 不了 車 ')],
 [(1.317937461058742, '個 小時 穿越 泰國 古今 了 解 泰國 豐厚 迷人 歷史 文化 '),
  (1.2605070080787228,
   '泰國 網卡 台灣 寄送 在 家 輕鬆 坐 泰國 網卡 TrueMove H 送上門 免費 贈送 sim卡 退卡針 '),
  (1.2579784139436474,
   '立即 預訂 泰國 曼谷 機場 接送 服務 往返 素 萬那普 

### Sentence Selection

In [12]:
from functools import reduce
num = 6 # choose top n sentences
sentence_select_list = []
for s in ranked_sentence_list:
    l = min(len(s),num)
    sentence_select = [y for (x,y) in s[0:l]]
    
    sentence_select = reduce(lambda x, y:str(x)+str(y),sentence_select)
    sentence_select_list.append(sentence_select)

In [13]:
result_PATH = './data/result/'
with open(result_PATH +'lexrank.json', 'w') as outfile:
    json.dump(sentence_select_list, outfile)

In [14]:
sentence_select_list

['胡志明市 出發 便利 接送 服務 你 少 交通 的 煩惱 越南  Viettel  上網 電話卡 前  8 GB  高速 上網 流量 超過 後 降速  64 kbps 效期  15 天 再 加贈 免費 接聽 來電 適用 越南 全 境 河內 胡志明市 重點 觀光區 皆 使用 ， 方便 旅途 中 隨時 卡 上網 來到 越南 胡志明市 預訂 印度 支那號 遊船 晚宴 共有 種 套餐 供 選擇 讓 在 欣賞 西貢河 沿岸 夜景 之餘 放鬆 享用 精緻 晚宴 美食 遊船 更 精彩 舞蹈 民謠 音樂 傳統 表演 帶給 胡志明 浪漫 夜晚 特別 體驗 想 越南 走跳 能 少 最 便宜 最 方便 背包客 必備 交通 方式  Open Bus 我們 搭上 小 船 入坐 馬車 到 濃郁 風情 湄公河 市鎮 一探 吧 椰林 遍佈 島 美托三角洲 離 胡志明市 段 距離 但 許多 遊客 深受 那裡 水鄉澤國 傳統 集市 的 吸引 願意 驅車 前往 ',
 '個 小時 穿越 泰國 古今 了 解 泰國 豐厚 迷人 歷史 文化 泰國 網卡 台灣 寄送 在 家 輕鬆 坐 泰國 網卡 TrueMove H 送上門 免費 贈送 sim卡 退卡針 立即 預訂 泰國 曼谷 機場 接送 服務 往返 素 萬那普 機場 BKK/ 廊曼 機場 DMK 曼谷 市區 曼谷 市區 學 正統 泰拳 2020 期間 限定 再 贈 4GB 20 分鐘 通話卡 8 天 GB 高流量 上網 通話 輕鬆 遊 泰國 提供 24 小時 兩 曼谷 機場 接送 優惠 連同 行李 一同 直達 曼谷 市區 飯店 ',
 '行程 東京 新宿 出發 登上 富士山 半山腰 五合目 在 河口湖 欣賞 富士山 絕 美 景致 10 個 小時 巴士 日 遊 帶 輕鬆 前往 富士山 周邊 人氣 景點 東京 新宿 出發 經典 富士山 日 遊 帶 一 睹 富士山 神秘 面紗 參加  KKday  富士山 箱根 日 遊 搭乘 舒適 專車 東京 出發 前往 東京 近郊 人氣 景點 一 覽 日本 神山 — 富士山 乘坐 經典 箱根 海盜船 遊 蘆之湖 享用 自助 日式 燒肉 火鍋 一 項 午餐  12 月  3 月 /  御殿場 燒烤 餐廳 享用 無限量 燒烤 午餐 4月 11月 ， 前往 富士山 二合目 滑雪場 體驗 玩 雪 雪橇 樂趣 12月 3月 /  富士